In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Load dataset
df = pd.read_csv('../datasets/50_Startups.csv')

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
# Separate features and target
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [ ]:
# Define categorical and numerical columns
cat_cols = [FIXME]
num_cols = [col for col in X.FIXME if col not in FIXME]

In [ ]:
sns.heatmap(X[FIXME].corr(), annot=True)

In [ ]:
# Preprocessor pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

numeric_transformer = Pipeline(steps=[
    ("imputer", FIXME(strategy="FIXME")),
])

categorical_transformer = FIXME(FIXME=[
    ("onehot", FIXME(handle_unknown="ignore", drop="FIXME"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, FIXME),
        ("cat", FIXME, FIXME)
    ]
)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
# Create a dictionary of models
models = {
    "model 1": FIXME(),
    "model 2": FIXME(random_state=42),
    "model 3": FIXME(random_state=42, verbosity=0)
}

In [ ]:
# Train-test split
from sklearn.model_selection import train_test_split
FIXME, FIXME, FIXME, FIXME = train_test_split(FIXME, FIXME, test_size=FIXME, random_state=14)

In [ ]:
# Run models
from sklearn.metrics import r2_score

print(f"{'Model':<20} {'Train R²':<10} {'Test R²':<10} {'Gap':<10}")
print("-" * 50)
for name, model in models.items():
    pipeline = Pipeline(steps=[
        ("preprocessor", FIXME),
        ("regressor", FIXME)
    ])
    FIXME.fit(X_train, y_train)
    train_preds = FIXME.predict(FIXME)
    test_preds = FIXME.predict(FIXME)
    
    train_score = FIXME(FIXME, FIXME)
    test_score = FIXME(y_test, FIXME)
    gap = train_score - test_score

    print(f"{name:<20} {train_score:<10.4f} {test_score:<10.4f} {gap:<10.4f}")